# Análisis Exploratorio de Muertes por COVID-19 en Estados Unidos

#### Este cuaderno es un análisis exploratorio de datos (EDA por sus siglas en inglés significa "Exploratory Data Analysis" ) sobre las muertes por COVID-19 en Estados Unidos. 

#### Se utilizarán diversas visualizaciones y técnicas para examinar y comprender los datos, buscando patrones interesantes y describiendo la historia que cuentan estos datos. 

# Librerías estándar de análisis de datos


In [14]:
import pandas as pd
import numpy as np

# Librerías de visualización

In [15]:
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool

# URL del dataset de muertes por COVID-19 en Estados Unidos

In [16]:
data_url = "https://api.covidtracking.com/v1/states/daily.json"

# Importar el dataset

In [17]:
df_death = pd.read_json(data_url)

# Visualización de las primeras filas del dataset con información de fecha y muertes

In [5]:
df_death['date'] = pd.to_datetime(df_death['date'], format='%Y%m%d')
df_death['death'] = df_death['death'].apply(lambda x: str(x).rstrip('.0'))
df_death['deathIncrease'] = df_death['deathIncrease'].apply(lambda x: 'NoIncrease' if x == 0 else x)
df_death[['date', 'state', 'death', 'deathIncrease']].sort_values(by='date', ascending=False).head(10)


,date,state,death,deathIncrease
0,2021-03-07,AK,305,NoIncrease
29,2021-03-07,MT,1381,NoIncrease
31,2021-03-07,ND,1478,NoIncrease
32,2021-03-07,NE,2113,NoIncrease
33,2021-03-07,NH,1184,3
34,2021-03-07,NJ,23574,17
35,2021-03-07,NM,3808,12
36,2021-03-07,NV,5037,1
37,2021-03-07,NY,39029,59
38,2021-03-07,OH,17656,NoIncrease


# Información básica del dataset

In [10]:
df_death.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20780 entries, 0 to 20779
Data columns (total 56 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         20780 non-null  datetime64[ns]
 1   state                        20780 non-null  object        
 2   positive                     20592 non-null  float64       
 3   probableCases                9271 non-null   float64       
 4   negative                     13290 non-null  float64       
 5   pending                      2138 non-null   float64       
 6   totalTestResultsSource       20780 non-null  object        
 7   totalTestResults             20614 non-null  float64       
 8   hospitalizedCurrently        17339 non-null  float64       
 9   hospitalizedCumulative       12382 non-null  float64       
 10  inIcuCurrently               11636 non-null  float64       
 11  inIcuCumulative              3789 non-nul

# Preparación de datos

## Convertir la columna 'date' al formato datetime


In [11]:
df_death['date'] = pd.to_datetime(df_death['date'], format='%Y%m%d')

# Descripción de las variables contenidas en el dataset:
#### Fecha: Fecha del registro.
#### Estado: Estado de Estados Unidos.
#### FIPS: Código de FIPS del estado.
#### Casos totales confirmados: Total de casos confirmados de COVID-19 en el estado.
#### Muertes totales: Total de muertes por COVID-19 en el estado.

## Análisis estadístico básic


In [7]:
df_death.describe()


,date,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,...,total,totalTestResultsIncrease,posNeg,dataQualityGrade,hospitalizedIncrease,commercialScore,negativeRegularScore,negativeScore,positiveScore,score
count,20780,2.059200e+04,9271.000000,1.329000e+04,2138.000000,2.061400e+04,17339.000000,12382.000000,11636.000000,3789.000000,...,2.078000e+04,20780.000000,2.078000e+04,0.0,20780.000000,20780.0,20780.0,20780.0,20780.0,20780.0
mean,2020-09-02 18:13:30.779595776,1.651560e+05,21729.123719,8.482246e+05,1659.862956,2.186936e+06,1190.576965,9262.762478,359.621176,1934.191607,...,7.063208e+05,17508.388210,7.061500e+05,NaN,37.360780,0.0,0.0,0.0,0.0,0.0
min,2020-01-13 00:00:00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,1.000000,0.000000,6.000000,...,0.000000e+00,-130545.000000,0.000000e+00,NaN,-12257.000000,0.0,0.0,0.0,0.0,0.0
25%,2020-06-02 00:00:00,5.753750e+03,773.500000,5.394125e+04,43.000000,1.040498e+05,166.500000,985.250000,60.000000,501.000000,...,2.577975e+04,1205.750000,2.565875e+04,NaN,0.000000,0.0,0.0,0.0,0.0,0.0
50%,2020-09-03 00:00:00,4.606450e+04,4491.000000,3.059720e+05,212.500000,6.552670e+05,531.000000,4472.000000,172.000000,1295.000000,...,1.937255e+05,6125.000000,1.935810e+05,NaN,0.000000,0.0,0.0,0.0,0.0,0.0
75%,2020-12-05 00:00:00,1.779580e+05,19553.000000,1.056611e+06,1295.750000,2.264766e+06,1279.000000,12248.500000,380.000000,2451.000000,...,7.737408e+05,19086.500000,7.737408e+05,NaN,36.000000,0.0,0.0,0.0,0.0,0.0
max,2021-03-07 00:00:00,3.501394e+06,365961.000000,1.018694e+07,64400.000000,4.964601e+07,22851.000000,82237.000000,5225.000000,9263.000000,...,1.125679e+07,473076.000000,1.124825e+07,NaN,16373.000000,0.0,0.0,0.0,0.0,0.0
std,NaN,3.267852e+05,45471.459778,1.344501e+06,4671.028548,4.436508e+06,2060.041207,12620.544081,594.831150,1953.329983,...,1.300339e+06,33586.022461,1.299997e+06,NaN,208.237151,0.0,0.0,0.0,0.0,0.0


# Número de estados distintos en el dataset


In [18]:
num_estados = df_death['state'].nunique()
print(f"Número de estados distintos en el dataset: {num_estados}")


Número de estados distintos en el dataset: 56


# Filtrar estados con un número insignificante de muertes para mejorar la visualización

In [19]:
min_deaths_threshold = 1000
df_filtered = df_death[df_death['death'] >= min_deaths_threshold]

# Obtener los 10 estados con más muertes

In [13]:
# Filtrar el DataFrame para eliminar las filas con valores no numéricos en la columna 'death'
df_death = df_death[pd.to_numeric(df_death['death'], errors='coerce').notnull()]

# Luego, continuar con el filtrado por umbral de muertes
min_deaths_threshold = 1000
df_filtered = df_death[df_death['death'].astype(float) >= min_deaths_threshold]


# Visualización de datos


## Gráfico de líneas del aumento diario de muertes por estado


In [20]:
output_notebook()

p = figure(x_axis_type="datetime", plot_width=800, plot_height=500, title="Aumento Diario de Muertes por COVID-19 en los 10 Estados con Más Muertes")
p.grid.grid_line_alpha = 0.3
p.xaxis.axis_label = 'Fecha'

for state in top_10_states:
    df_state = df_death[df_death['state'] == state]
    p.line(df_state['date'], df_state['death'], line_width=2, legend_label=state)

p.legend.location = "top_left"
p.legend.title = 'Estado'
show(p)

Loading BokehJS ...

AttributeError: unexpected attribute 'plot_width' to figure, similar attributes are outer_width, width or min_width